In [1]:
# Import sys, then tell python where to find the nEXO-specific classes
# I'll assume you're in the sensitivity/work directory, as shown here by !pwd

import sys
!pwd
sys.path.append('../../modules')

/Users/czyz1/lc-home/nexo_sensitivity/work/SensitivityPaper2020_scripts


In [2]:
# Import useful libraries for analysis

import pandas as pd
import histlite as hl
import numpy as np
from matplotlib import pyplot as plt
import os

import matplotlib
from matplotlib.backends.backend_pdf import PdfPages

plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (10,8)

In [22]:
# Import the nEXO sensitivity classes

plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (10,8)

import nEXOFitWorkspace
import nEXOFitModel
import nEXOFitLikelihood
import nEXOMaterialsDBInterface

import importlib
importlib.reload( nEXOMaterialsDBInterface )
importlib.reload( nEXOFitWorkspace )
importlib.reload( nEXOFitModel )

path_home = '/Users/czyz1/lc-home/nexo_sensitivity/work'
path_result = '/Users/czyz1/lc-nexouser'
config_loc = "{}/config/Sensitivity2020_Optimized_DNN_Standoff_Binning_version1.yaml".format(path_home)
date = '21_01_13'
database_num = '023'
resolution = '0.008'

# Bkg contributions from baseline 2019 (D-023)

In [24]:
#optimized_binning_yaml = '/p/vast1/nexo/sensitivity2020/pdfs/config_files/Sensitivity2020_Optimized_DNN_Standoff_Binning_version1.yaml'
optimized_binning_yaml = config_loc
workspace2020 = nEXOFitWorkspace.nEXOFitWorkspace(config=optimized_binning_yaml)
workspace2020_ind = nEXOFitWorkspace.nEXOFitWorkspace(config=optimized_binning_yaml)


optimized_binning_components_table = "{}/workdir/components_tables/{}/ComponentsTable_D-{}_Energy_Res={}.h5".\
        format(path_result, date, database_num, resolution)
# optimized_binning_components_table = '/p/vast1/nexo/sensitivity2020/pdfs/component_tables/ComponentsTable_D-023_merged-v10b_Optimized_DNN_Standoff_Binning_version1.h5'
workspace2020.LoadComponentsTableFromFile(optimized_binning_components_table)
workspace2020_ind.LoadComponentsTableFromFile(optimized_binning_components_table)




Loading input data froma previously-generated components table....

Loaded dataframe with 166 components.
Contains the following quantities of interest:
	PDFName
	Component
	Isotope
	MC ID
	Total Mass or Area
	Activity ID
	SpecActiv
	SpecActivErr
	SpecActivErrorType
	RawActiv
	RawActivErr
	TotalHitEff_N
	Histogram
	HistogramAxisNames
	TotalHitEff_K
	Group
	Expected Counts

Fit variables:
	['DNN', 'Energy (keV)', 'Standoff (mm)']

Loading input data froma previously-generated components table....

Loaded dataframe with 166 components.
Contains the following quantities of interest:
	PDFName
	Component
	Isotope
	MC ID
	Total Mass or Area
	Activity ID
	SpecActiv
	SpecActivErr
	SpecActivErrorType
	RawActiv
	RawActivErr
	TotalHitEff_N
	Histogram
	HistogramAxisNames
	TotalHitEff_K
	Group
	Expected Counts

Fit variables:
	['DNN', 'Energy (keV)', 'Standoff (mm)']


In [28]:
#Give each compoennt its own group

groupdict2020 = {}
for index, row in workspace2020_ind.df_components.iterrows():
    if row['Histogram'] is None:
        continue
    groupdict2020[row['PDFName']] = row['Group']
    workspace2020_ind.df_components.loc[index,'Group'] = row['PDFName']



In [32]:
groups = set(groupdict2020.values())

group_component_dict = dict()

for group in groups:
    group_component_dict[group] = [k for k,v in groupdict2020.items() if v == group]


['Th232_OuterCryostatFeedthrough']
['U238_OuterCryostatSupportFiber']
['Rn222_InactiveLXe']
['Th232_Cathode']
['K40_OuterCryostatSupportResin']
['Th232_ChargeTilesElectronics']
['Th232_HVFeedthrough']
['Co60_TPCSupportCone']
['U238_SupportRodsandSpacers']
['Th232_ChargeTilesBacking']
['K40_OuterCryostatFeedthrough']
['Co60_HVTubes']
['bb0n_FullLXe']
['U238_InnerCryostatSupportFiber']
['Co60_InnerCryostatSupportFiber']
['Th232_SiPMModuleInterposer']
['U238_ChargeTilesSupport']
['Co60_CalibrationGuideTube1']
['bb2n_FullLXe']
['U238_InnerCryostatFeedthrough']
['Co60_TPCVessel']
['Xe137_ActiveLXe']
['Th232_SupportRodsandSpacers']
['U238_HFE']
['K40_HVTubes']
['Th232_CalibrationGuideTube2']
['B8nu_FullLXe']
['U238_InnerCryostatSupportResin']
['U238_ChargeTilesElectronics']
['Th232_ElectricalConnectionsSiPM']
['U238_OuterCryostatResin']
['U238_SiPMs']
['K40_OuterCryostatFiber']
['Co60_InnerCryostatResin']
['Th232_TPCVessel']
['Th232_HVFeedthroughCoreCable']
['Co60_HVFeedthrough']
['K40_HVPlu

In [31]:
#Create the PDFs for the standard model
workspace2020.CreateGroupedPDFs()
model2020 = nEXOFitModel.nEXOFitModel()
model2020.AddPDFsFromDataframe(workspace2020.df_group_pdfs,\
                               workspace2020.histogram_axis_names)
model2020.GenerateModelDistribution()




Creating grouped PDFs....


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
# Create the PDFs for the individual-component model
workspace2020_ind.CreateGroupedPDFs()
model2020_ind = nEXOFitModel.nEXOFitModel()
model2020_ind.AddPDFsFromDataframe(workspace2020_ind.df_group_pdfs,\
                               workspace2020_ind.histogram_axis_names)
model2020_ind.GenerateModelDistribution()




In [9]:
roi_cut_dict_2020 = {'DNN': (0.86,1.),
                    'Energy (keV)': (2434., 2480.),
                    'Standoff (mm)': (104.5, 650.)
                    }

workspace2020.DefineROI( roi_cut_dict_2020 )
workspace2020_ind.DefineROI( roi_cut_dict_2020 )


TypeError: argument of type 'NoneType' is not iterable

## Relative contributions of each group to ROI

In [ ]:
total_in_ROI = model2020.GetIntegralInBinRange(workspace2020.GetROIBinIndices())
print('{:4.4} total events in ROI'.format(total_in_ROI))

print('{:<20} {:>13} {:>13}'.format('Name','Counts','Percent'))

group_bkg_dict = dict()

for row in model2020.variable_list:
    this_group_in_roi = model2020.GetComponentIntegralInBinRange(row['Name'],workspace2020.GetROIBinIndices())
    print('{:<20} {:>13.3} {:>13.3}%'.format(row['Name'],\
                                            this_group_in_roi,\
                                            this_group_in_roi/total_in_ROI*100.))
    group_bkg_dict[row['Name'][4:]] = this_group_in_roi



In [ ]:
# Name                        Counts       Percent
# Num_Far                       1.09          19.2%
# Num_Vessel_U238              0.358          6.26%
# Num_Vessel_Th232            0.0887          1.55%
# Num_Internals_U238           0.833          14.6%
# Num_Internals_Th232          0.061          1.07%
# Num_FullTPC_K40                0.0           0.0%
# Num_Rn222                     2.75          48.1%
# Num_FullLXeBb2n             0.0604          1.06%
# Num_FullLXeBb0n           3.99e-05      0.000699%
# Num_B8nu                     0.118          2.07%
# Num_Xe137                    0.353          6.18%

## Looking at individual groups

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (10,8)


spectrum_cut_dict_2020 = {'DNN': (0.86,1.),
               'Energy (keV)': (1000.,3500.),
               'Standoff (mm)': (100.,650.)
              }

counter = 0
for group in groups:
    
    plt.figure(counter)
    
    if group == 'Off': continue
    
    component_list = group_component_dict[group]
    
    # Get first component histogram, so we can get the shape and create an empty sum_hist
    idx = model2020_ind.GetVariableIndexByName(component_list[0])
    test_hist = model2020_ind.GetSlicedDistribution(cut_dict=spectrum_cut_dict_2020,\
                                                    renormalize=False,\
                                                    var_name=component_list[0],\
                                                    verbose=False)
    sum_hist = hl.Hist(bins=test_hist.bins,values=np.zeros(test_hist.values.shape))
    
    print('\n********** {} ***********'.format(group))
    print('{:<33} {:>13} {:>16}'.format('Name','Counts (ROI)','Percent of grp'))
    
    

    for component in component_list:
        this_hist = model2020_ind.GetSlicedDistribution(cut_dict=spectrum_cut_dict_2020,\
                                                renormalize=False,\
                                                var_name=component,\
                                                verbose=False)
        idx = model2020_ind.GetVariableIndexByName(component)
        this_val = model2020_ind.variable_list[idx]['Value']
        hl.plot1d(this_hist.project([1])*this_val,label=component)
        
        sum_hist = sum_hist + this_hist*this_val
        
        this_comp_in_roi = model2020_ind.GetComponentIntegralInBinRange(component,\
                                                    workspace2020_ind.GetROIBinIndices())
        print('{:<33} {:>13.3} {:>16.4}%'.format(component,\
                                            this_comp_in_roi,\
                                            this_comp_in_roi/group_bkg_dict[group]*100.))
        
        
    hl.plot1d(sum_hist.project([1]),color='k',linewidth=2,label="Sum")
    plt.yscale('log')
    plt.legend(fontsize=8)
    plt.ylabel('Single-site counts in 10yr in inner ~2T')
    plt.xlabel('Energy (keV)')
    plt.title(group)
    plt.xlim(1000.,3500.)
        
    
    
    counter+=1
    
    
    

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (10,8)


spectrum_cut_dict_2020 = {'DNN': (0.86,1.),
               'Energy (keV)': (1000.,3500.),
               'Standoff (mm)': (100.,650.)
              }

counter = 0
for group in groups:

    if 'Rn222' not in group: continue
    
    
    plt.figure(counter)

    #if group == 'Off': continue
    
    component_list = group_component_dict[group]
    
    # Get first component histogram, so we can get the shape and create an empty sum_hist
    idx = model2020_ind.GetVariableIndexByName(component_list[0])
    test_hist = model2020_ind.GetSlicedDistribution(cut_dict=spectrum_cut_dict_2020,\
                                                    renormalize=False,\
                                                    var_name=component_list[0],\
                                                    verbose=False)
    sum_hist = hl.Hist(bins=test_hist.bins,values=np.zeros(test_hist.values.shape))
    
    print('\n********** {} ***********'.format(group))
    print('{:<33} {:>13} {:>16}'.format('Name','Counts (ROI)','Percent of grp'))
    
    

    for component in component_list:
        this_hist = model2020_ind.GetSlicedDistribution(cut_dict=spectrum_cut_dict_2020,\
                                                renormalize=False,\
                                                var_name=component,\
                                                verbose=False)
        idx = model2020_ind.GetVariableIndexByName(component)
        this_val = model2020_ind.variable_list[idx]['Value']
        hl.plot1d(this_hist.project([1])*this_val,label=component)
        
        sum_hist = sum_hist + this_hist*this_val
        
        this_comp_in_roi = model2020_ind.GetComponentIntegralInBinRange(component,\
                                                    workspace2020_ind.GetROIBinIndices())
        print('{:<33} {:>13.3} {:>16.4}%'.format(component,\
                                            this_comp_in_roi,\
                                            this_comp_in_roi/group_bkg_dict[group]*100.))
        
    # Now look at all "off" components    
    component_list = group_component_dict['Off']
    
    for component in component_list:
        if 'Rn222' not in component: continue
        this_hist = model2020_ind.GetSlicedDistribution(cut_dict=spectrum_cut_dict_2020,\
                                                renormalize=False,\
                                                var_name=component,\
                                                verbose=False)
        idx = model2020_ind.GetVariableIndexByName(component)
        this_val = model2020_ind.variable_list[idx]['Value']
        hl.plot1d(this_hist.project([1])*this_val,label=component)
        
        #sum_hist = sum_hist + this_hist*this_val
        
        this_comp_in_roi = model2020_ind.GetComponentIntegralInBinRange(component,\
                                                    workspace2020_ind.GetROIBinIndices())
        print('{:<33} {:>13.3} {:>16.4}%'.format(component,\
                                            this_comp_in_roi,\
                                            this_comp_in_roi/group_bkg_dict[group]*100.))
        
        
    hl.plot1d(sum_hist.project([1]),color='k',linewidth=2,label="Sum")
    plt.yscale('log')
    plt.legend(fontsize=10)
    plt.ylabel('Single-site counts in 10yr in inner ~2T')
    plt.xlabel('Energy (keV)')
    plt.title(group)
    plt.xlim(1000.,3500.)
        
    
    
    counter+=1
    

# Bkg contributions from electroformed copper model (D-024)




In [10]:
optimized_binning_yaml = '../config/Sensitivity2020_Optimized_DNN_Standoff_Binning_version1.yaml'
workspace2020_d024 = nEXOFitWorkspace.nEXOFitWorkspace(config=optimized_binning_yaml)
workspace2020_d024_ind = nEXOFitWorkspace.nEXOFitWorkspace(config=optimized_binning_yaml)

optimized_binning_components_table = '/p/vast1/nexo/sensitivity2020/pdfs/component_tables/ComponentsTable_D-024_merged-v10b_Optimized_DNN_Standoff_Binning_version1.h5'
workspace2020_d024.LoadComponentsTableFromFile(optimized_binning_components_table)
workspace2020_d024_ind.LoadComponentsTableFromFile(optimized_binning_components_table)





Loading input data froma previously-generated components table....

ERROR: The input file must be an HDF5 file.

File /p/vast1/nexo/sensitivity2020/pdfs/component_tables/ComponentsTable_D-024_merged-v10b_Optimized_DNN_Standoff_Binning_version1.h5 does not exist

Loading input data froma previously-generated components table....

ERROR: The input file must be an HDF5 file.

File /p/vast1/nexo/sensitivity2020/pdfs/component_tables/ComponentsTable_D-024_merged-v10b_Optimized_DNN_Standoff_Binning_version1.h5 does not exist


In [11]:
#Give each compoennt its own group

groupdict2020_d024 = {}
for index, row in workspace2020_d024_ind.df_components.iterrows():
    if row['Histogram'] is None:
        continue
    groupdict2020_d024[row['PDFName']] = row['Group']
    workspace2020_d024_ind.df_components.loc[index,'Group'] = row['PDFName']

groups_d024 = set(groupdict2020_d024.values())

group_component_dict_d024 = dict()

for group in groups_d024:
    group_component_dict_d024[group] = \
                [k for k,v in groupdict2020_d024.items() if v == group]
    


In [12]:
#Create the PDFs for the standard model
workspace2020_d024.CreateGroupedPDFs()
model2020_d024 = nEXOFitModel.nEXOFitModel()
model2020_d024.AddPDFsFromDataframe(workspace2020_d024.df_group_pdfs,\
                               workspace2020_d024.histogram_axis_names)
model2020_d024.GenerateModelDistribution()

# Create the PDFs for the individual-component model
workspace2020_d024_ind.CreateGroupedPDFs()
model2020_d024_ind = nEXOFitModel.nEXOFitModel()
model2020_d024_ind.AddPDFsFromDataframe(workspace2020_d024_ind.df_group_pdfs,\
                               workspace2020_d024_ind.histogram_axis_names)
model2020_d024_ind.GenerateModelDistribution()



Creating grouped PDFs....
	Group:                              	Expected Counts:
	nan                                 	            nan

Creating grouped PDFs....
	Group:                              	Expected Counts:
	nan                                 	            nan


nan

## Relative contributions to ROI

In [13]:
roi_cut_dict_2020_d024 = {'DNN': (0.86,1.),
                    'Energy (keV)': (2434., 2480.),
                    'Standoff (mm)': (104.5, 650.)
                    }

workspace2020_d024.DefineROI( roi_cut_dict_2020_d024 )
workspace2020_d024_ind.DefineROI( roi_cut_dict_2020_d024 )


total_in_ROI_d024 = model2020_d024.GetIntegralInBinRange(workspace2020_d024.GetROIBinIndices())
print('\n\n{:4.4} total events in ROI'.format(total_in_ROI_d024))

print('{:<20} {:>13} {:>13}'.format('Name','Counts','Percent'))

group_bkg_dict_d024 = dict()

for row in model2020_d024.variable_list:
    this_group_in_roi = model2020_d024.GetComponentIntegralInBinRange(row['Name'],workspace2020_d024.GetROIBinIndices())
    print('{:<20} {:>13.3} {:>13.3}%'.format(row['Name'],\
                                            this_group_in_roi,\
                                            this_group_in_roi/total_in_ROI_d024*100.))
    group_bkg_dict_d024[row['Name'][4:]] = this_group_in_roi

TypeError: argument of type 'NoneType' is not iterable

In [14]:
# plt.rcParams.update({'font.size': 18})
# plt.rcParams['figure.figsize'] = (10,8)


# spectrum_cut_dict_2020 = {'DNN': (0.86,1.),
#                'Energy (keV)': (1000.,3500.),
#                'Standoff (mm)': (100.,650.)
#               }

# counter = 0
# for group in groups_d024:
    
#     plt.figure(counter)
    
#     if group == 'Off': continue
    
#     component_list = group_component_dict_d024[group]
    
#     # Get first component histogram, so we can get the shape and create an empty sum_hist
#     idx = model2020_d024_ind.GetVariableIndexByName(component_list[0])
#     test_hist = model2020_d024_ind.GetSlicedDistribution(cut_dict=spectrum_cut_dict_2020,\
#                                                     renormalize=False,\
#                                                     var_name=component_list[0],\
#                                                     verbose=False)
#     sum_hist = hl.Hist(bins=test_hist.bins,values=np.zeros(test_hist.values.shape))
    
#     print('\n********** {} ***********'.format(group))
#     print('{:<33} {:>13} {:>16}'.format('Name','Counts (ROI)','Percent of grp'))
    
    

#     for component in component_list:
#         this_hist = model2020_d024_ind.GetSlicedDistribution(cut_dict=spectrum_cut_dict_2020,\
#                                                 renormalize=False,\
#                                                 var_name=component,\
#                                                 verbose=False)
#         idx = model2020_d024_ind.GetVariableIndexByName(component)
#         this_val = model2020_d024_ind.variable_list[idx]['Value']
#         hl.plot1d(this_hist.project([1])*this_val,label=component)
        
#         sum_hist = sum_hist + this_hist*this_val
        
#         this_comp_in_roi = model2020_d024_ind.GetComponentIntegralInBinRange(component,\
#                                                     workspace2020_d024_ind.GetROIBinIndices())
#         print('{:<33} {:>13.3} {:>16.4}%'.format(component,\
#                                             this_comp_in_roi,\
#                                             this_comp_in_roi/group_bkg_dict_d024[group]*100.))
        
        
#     hl.plot1d(sum_hist.project([1]),color='k',linewidth=2,label="Sum")
#     plt.yscale('log')
#     plt.legend(fontsize=8)
#     plt.ylabel('Single-site counts in 10yr in inner ~2T')
#     plt.xlabel('Energy (keV)')
#     plt.title(group)
#     plt.xlim(1000.,3500.)
        
    
    
#     counter+=1
    

## Background comparison for D-023, D-024

In [15]:
colors=plt.rcParams['axes.prop_cycle'].by_key()['color']



last_group_counts_023 = 0.
last_group_counts_024 = 0.

counter = 0
facealpha = 1.
for name,counts in group_bkg_dict.items():
    
    if name != 'FullTPC_Co60':
        print('{:<16} {:>10.4} {:>10.4}'.format(name,counts,group_bkg_dict_d024[name]))
    
    if counter == len(colors):
        counter = 0
        facealpha*=0.5
    
    plt.bar(1,counts,0.5,bottom=last_group_counts_023,label=name,color=colors[counter],alpha=facealpha)
    last_group_counts_023 += counts
    
    if name == 'FullTPC_Co60':
        plt.bar(2,0.,0.5,bottom=last_group_counts_024,color=colors[counter],alpha=facealpha)
    else:
        plt.bar(2,group_bkg_dict_d024[name],0.5,bottom=last_group_counts_024,color=colors[counter],alpha=facealpha)
        last_group_counts_024 += group_bkg_dict_d024[name]
        
    counter += 1
        
plt.legend(fontsize=12)
plt.xlim(0.5,3.)
plt.ylabel('Expected counts in ROI')
plt.xticks([1.,2.],['Normal Copper\n(D-023)','Electroformed Copper\n(D-024)'])
plt.savefig('plots/roi_background_contributions_by_group_D023_and_D024.png',dpi=200,bbox_inches='tight')


    

NameError: name 'group_bkg_dict' is not defined

# Comparing the Rn222 PDF with the Internals_U238 PDF

In [16]:
int_idx = model2020.GetVariableIndexByName('Internals_U238')
rn222_idx = model2020.GetVariableIndexByName('Rn222')



ValueError: No variable in the likelihood.variable_list contains the name Internals_U238.

	Please double-check that the variable names in the ComponentsTable match
	the ones you're trying to access



In [ ]:
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (8,6)

histint = model2020.pdfs[int_idx]


hl.plot2d(histint.project([0,1]).log10())
plt.xlabel('DNN')
plt.ylabel('Energy (keV)')


plt.figure(2)
hl.plot2d(histint.project([1,2]).log10())
plt.xlabel('Energy (keV)')
plt.ylabel('Standoff (mm)')

plt.figure(3)
hl.plot2d(histint.project([0,2]).log10())
plt.xlabel('DNN')
plt.ylabel('Standoff (mm)')


In [17]:
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (8,6)

histrn222 = model2020.pdfs[rn222_idx]


hl.plot2d(histrn222.project([0,1]).log10())
plt.xlabel('DNN')
plt.ylabel('Energy (keV)')


plt.figure(2)
hl.plot2d(histrn222.project([1,2]).log10())
plt.xlabel('Energy (keV)')
plt.ylabel('Standoff (mm)')

plt.figure(3)
hl.plot2d(histrn222.project([0,2]).log10())
plt.xlabel('DNN')
plt.ylabel('Standoff (mm)')

NameError: name 'rn222_idx' is not defined

In [18]:
eproj_cut_dict = {'DNN': (0.86,1.),
                    'Energy (keV)': (1000., 3500.),
                    'Standoff (mm)': (104.5, 650.)
                    }
stproj_cut_dict = {'DNN': (0.86,1.),
                    'Energy (keV)': (2434., 2480.),
                    'Standoff (mm)': (0., 650.)
                    }
dnn_cut_dict = {'DNN': (0.,1.),
                    'Energy (keV)': (2434., 2480.),
                    'Standoff (mm)': (104.5, 650.)
                    }

rn222_in_roi = model2020.GetComponentIntegralInBinRange('Rn222',\
                                                        workspace2020.GetROIBinIndices())
rn222_idx = model2020.GetVariableIndexByName('Rn222')
int_in_roi = model2020.GetComponentIntegralInBinRange('Internals_U238',\
                                                        workspace2020.GetROIBinIndices())
int_idx = model2020.GetVariableIndexByName('Internals_U238')
bb0n_in_roi = model2020.GetComponentIntegralInBinRange('Bb0n',\
                                                        workspace2020.GetROIBinIndices())
bb0n_idx = model2020.GetVariableIndexByName('Bb0n')

rn222_scale = model2020.variable_list[rn222_idx]['Value']/rn222_in_roi
int_scale = model2020.variable_list[int_idx]['Value']/int_in_roi
bb0n_scale = model2020.variable_list[bb0n_idx]['Value']/bb0n_in_roi


# Get the energy projections
rn222eproj = model2020.GetSlicedDistribution(cut_dict=eproj_cut_dict,\
                                                renormalize=False,\
                                                var_name='Rn222',\
                                                verbose=False)
inteproj = model2020.GetSlicedDistribution(cut_dict=eproj_cut_dict,\
                                                renormalize=False,\
                                                var_name='Internals_U238',\
                                                verbose=False)
bb0neproj = model2020.GetSlicedDistribution(cut_dict=eproj_cut_dict,\
                                                renormalize=False,\
                                                var_name='Bb0n',\
                                                verbose=False)
#model2020.GetComponentIntegralInBinRange()
hl.plot1d( rn222eproj.project([1])*rn222_scale, color='b', label='Rn222')
hl.plot1d( inteproj.project([1])*int_scale, color='r', label='Internals_U238')
hl.plot1d( bb0neproj.project([1])*bb0n_scale,color='g', label='Bb0n')

plt.xlabel('Energy (keV)')
plt.ylabel('Normalized to counts in ROI')
plt.title('Energy spectrum for DNN>0.86 in inner 2T',fontsize=16)
plt.legend(fontsize=16)
plt.xlim(2000.,3000.)
#plt.yscale('log')
plt.ylim(0.,1.)

plt.savefig('plots/rn222_internalsU238_energy_spectrum_comparison.png',\
            dpi=200,bbox_inches='tight')

plt.figure(2)

rn222stproj = model2020.GetSlicedDistribution(cut_dict=stproj_cut_dict,\
                                                renormalize=False,\
                                                var_name='Rn222',\
                                                verbose=False)
intstproj = model2020.GetSlicedDistribution(cut_dict=stproj_cut_dict,\
                                                renormalize=False,\
                                                var_name='Internals_U238',\
                                                verbose=False)
bb0nstproj = model2020.GetSlicedDistribution(cut_dict=stproj_cut_dict,\
                                                renormalize=False,\
                                                var_name='Bb0n',\
                                                verbose=False)
hl.plot1d( rn222stproj.project([2])*rn222_scale, color='b', label='Rn222')
hl.plot1d( intstproj.project([2])*int_scale, color='r', label='Internals_U238')
hl.plot1d( bb0nstproj.project([2])*bb0n_scale,color='g', label='Bb0n')

plt.xlabel('Standoff (mm)')
plt.ylabel('Normalized to counts in ROI')
plt.title('Standoff for DNN>0.86 and energy between 2430-2480 keV',fontsize=16)
plt.legend(fontsize=16)
#plt.yscale('log')
plt.ylim(0.,1.)
plt.savefig('plots/rn222_internalsU238_standoff_comparison.png',\
            dpi=200,bbox_inches='tight')


plt.figure(3)

rn222dnnproj = model2020.GetSlicedDistribution(cut_dict=dnn_cut_dict,\
                                                renormalize=False,\
                                                var_name='Rn222',\
                                                verbose=False)
intdnnproj = model2020.GetSlicedDistribution(cut_dict=dnn_cut_dict,\
                                                renormalize=False,\
                                                var_name='Internals_U238',\
                                                verbose=False)
bb0ndnnproj = model2020.GetSlicedDistribution(cut_dict=dnn_cut_dict,\
                                                renormalize=False,\
                                                var_name='Bb0n',\
                                                verbose=False)
hl.plot1d( rn222dnnproj.project([0])*rn222_scale, color='b', label='Rn222')
hl.plot1d( intdnnproj.project([0])*int_scale, color='r', label='Internals_U238')
hl.plot1d( bb0ndnnproj.project([0])*bb0n_scale,color='g', label='Bb0n')

plt.xlabel('DNN')
plt.ylabel('Normalized to counts in ROI')
plt.title('DNN for standoff > 100mm and energy between 2430-2480 keV',fontsize=16)
plt.legend(fontsize=16)
#plt.yscale('log')
plt.ylim(0.,1.)
plt.savefig('plots/rn222_internalsU238_DNN_comparison.png',\
            dpi=200,bbox_inches='tight')






TypeError: object of type 'NoneType' has no len()